In [3]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [4]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
data = pd.read_sql_query('SELECT * FROM film', engine)
data.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [5]:
query = '''select f.film_id, f.length, f.release_year, f.rental_duration, f.rental_rate, f.rating, c.category_id as category
from sakila.film f
join sakila.film_category fc
on f.film_id = fc.film_id
join sakila.category c
on fc.category_id = c.category_id
where c.category_id < 5;'''
data = pd.read_sql_query(query, engine)
data.head()

,film_id,length,release_year,rental_duration,rental_rate,rating,category
0,19,113,2006,6,0.99,PG,1
1,21,129,2006,3,4.99,R,1
2,29,168,2006,5,2.99,NC-17,1
3,38,68,2006,6,0.99,NC-17,1
4,56,129,2006,6,2.99,G,1


In [ ]:
data['category'].value_counts()

In [ ]:
data.shape

In [ ]:
data['release_year'].value_counts()

In [6]:
data = data.drop(columns=['release_year']) #all release_years = 2006

In [7]:
data = data.drop(columns= ['film_id']) #unique identifier 

In [ ]:
data.head()

In [ ]:
data['length'].value_counts()

In [ ]:
data['rental_rate'].value_counts()

In [ ]:
data['rental_duration'].value_counts()

In [8]:
data.dtypes

length               int64
rental_duration      int64
rental_rate        float64
rating              object
category             int64
dtype: object

In [ ]:
data.isna().sum()

In [9]:
data['rental_duration'] = data['rental_duration'].astype('object') 

In [ ]:
#data['category'] = data['category'].astype('object')

In [10]:
data['rental_rate'] = data['rental_rate'].astype('object') 

In [11]:
data.describe()

,length,category
count,247.000000,247.000000
mean,111.024291,2.445344
std,40.311281,1.109845
min,46.000000,1.000000
25%,78.500000,1.000000
50%,108.000000,2.000000
75%,143.000000,3.000000
max,185.000000,4.000000


In [ ]:
"""def cat_length(minutes):
    if minutes < 90:
        return "short"
    if minutes > 140:
        return "long"
    else:
        return "medium""""

In [ ]:
#data['length'] = list(map(cat_length, data['length']))

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
"""for col in ['rental_duration', 'rental_rate']:
    data[col] = pd.to_numeric(data[col])"""

In [ ]:
data.dtypes

In [ ]:
#data['category'] = data['category'].astype('object') 

In [ ]:
data.dtypes

In [12]:
from sklearn.preprocessing import Normalizer
# from sklearn.preprocessing import StandardScaler
import numpy as np
X = data.select_dtypes(include = np.number)

# Normalizing data
transformer = Normalizer().fit(X)
x_normalized = transformer.transform(X)
x = pd.DataFrame(x_normalized)

In [ ]:
x

In [14]:
cat = data.select_dtypes(include = np.object)
#cat = cat.drop(['category'], axis=1)
categorical = pd.get_dummies(cat, columns=['rental_duration', 'rental_rate', 'rating'])

In [15]:
y = data['category']

In [16]:
X = np.concatenate((x, categorical), axis=1)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

In [18]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train, y_train)

In [19]:
classification.score(X_test, y_test)

0.1919191919191919

In [20]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.1919191919191919

In [21]:
pd.Series(predictions).value_counts()

1    39
3    30
2    24
4     6
dtype: int64

In [22]:
y_test.value_counts()

4    28
2    25
1    24
3    22
Name: category, dtype: int64

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[ 9,  6,  8,  1],
       [ 8,  6,  9,  2],
       [11,  4,  4,  3],
       [11,  8,  9,  0]])